In [4]:
import os
import boto3
import subprocess

In [5]:
### initialization ###
ec2_client = boto3.client("ec2", region_name="ap-southeast-1")
session = boto3.Session(region_name='ap-southeast-1')
ec2_resource = session.resource('ec2')

### config ###
key_pair_name = "ec2-key-pair"
instance = "r5.2xlarge"
image_id = 'ami-067a3008b35fd1a98' 
elastic_ip = '13.251.215.161'
instance_id = 'i-03ebe313c35226f1a'
storage_size_gb = 16
mode = 'stop'

### config 2
# key_pair_name = "ec2-key-pair"
# instance = "p3.2xlarge"
# image_id = 'ami-067a3008b35fd1a98' 
# elastic_ip = ''
# instance_id = 'i-0c4fe01e137da5af6'
# storage_size_gb = 16
# mode = 'stop'

In [6]:
if mode == 'create':
    ### 1. creating keypair ###
    def create_key_pair(name):
        key_pair = ec2_client.create_key_pair(KeyName=name)
        private_key = key_pair["KeyMaterial"]
        # write private key to file with 400 permissions
        file_path = os.path.expanduser(f"~/.ssh/{name}.pem")
        with os.fdopen(os.open(file_path, os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle:
            handle.write(private_key)

    # to check if keypair exists
    response = ec2_client.describe_key_pairs()
    key_pairs = response['KeyPairs']
    key_pair_exists = any(key_pair['KeyName'] == key_pair_name for key_pair in key_pairs)
    if not key_pair_exists:
        create_key_pair(key_pair_name)
        # key_pair = ec2_resource.KeyPair(key_pair_name)
        # key_pair.delete()

    ### 2. create security group ###
    try:
        security_group = ec2_resource.create_security_group(
            GroupName='ec2_security_sg',
            Description='ec2 security group for SSH access'
        )

        # Authorize inbound SSH traffic to the security group
        security_group.authorize_ingress(
            IpProtocol='tcp',
            FromPort=22,
            ToPort=22,
            CidrIp='0.0.0.0/0'
        )
        security_group_name = security_group.group_name
    except:
        # if security group exists then search for it
        response = ec2_client.describe_security_groups(
            Filters=[
                {
                    'Name': 'group-name',
                    'Values': ['ec2_security_sg']
                }
            ])
        security_groups = response['SecurityGroups']
        security_group = security_groups[0]
        security_group_name = security_group['GroupName']

    ### 3. creating new pytorch instance ###
    def create_instance(key_name, instance='g4dn.2xlarge', ami_id='ami-067a3008b35fd1a98'):
        instances = ec2_client.run_instances(
            ImageId=ami_id,
            MinCount=1,
            MaxCount=1,
            InstanceType=instance,
            KeyName=key_name,
            SecurityGroups=[security_group_name],
            BlockDeviceMappings=[
            {
                'DeviceName': '/dev/xvdb', 
                'Ebs': {
                    'VolumeSize': storage_size_gb,
                    'VolumeType': 'gp2',  
                },
            },
        ])
        
        instance_id = instances["Instances"][0]["InstanceId"]
        waiter = ec2_client.get_waiter('instance_running')
        waiter.wait(InstanceIds=[instance_id])
        return instance_id

    instance_id = create_instance(key_pair_name, instance, image_id)
    print("Instance created: ", instance_id)

    ### 4. create elastic ip ###
    # if elastic_ip == '':
    #     response = ec2_client.allocate_address(Domain='vpc')
    #     elastic_ip = response['PublicIp']
    #     tag_key = 'name'
    #     tag_value = 'ec2_elastic_ip'
    #     # Add tags to the Elastic IP address
    #     ec2_client.create_tags(
    #         Resources=[response['AllocationId']],
    #         Tags=[{'Key': tag_key, 'Value': tag_value}])
    #     print("Elastic IP address created and tagged:", elastic_ip)

    if elastic_ip != '':
        # Associate the Elastic IP with the EC2 instance
        response = ec2_client.associate_address(
            InstanceId=instance_id,
            PublicIp=elastic_ip,
        )

elif mode == 'start':
    ### to start instance ###
    response = ec2_client.start_instances(InstanceIds=[instance_id])
    instance = ec2_resource.Instance(instance_id)
    instance.wait_until_running()
    instance_status = instance.state['Name']
    print(instance_status)

elif mode == 'stop':
    ### shut down instance ###
    response = ec2_client.stop_instances(InstanceIds=[instance_id])
    instance = ec2_resource.Instance(instance_id)
    instance_status = instance.state['Name']
    print(instance_status)

elif mode == 'kill':
    ### kill instance ###
    response = ec2_client.terminate_instances(InstanceIds=[instance_id])
    print(response)

### get ip address of the instance ###
def get_public_ip(instance_id):
    reservations = ec2_client.describe_instances(InstanceIds=[instance_id]).get("Reservations")
    for reservation in reservations:
        for instance in reservation['Instances']:
            print(instance.get("PublicIpAddress"))
get_public_ip(instance_id)

stopping
13.251.215.161


In [7]:
### check running instance ###
response = ec2_client.describe_instances(
    Filters=[
        {
            'Name': 'instance-state-name',
            'Values': ['running']
        }
    ]
)
running_instances = []
for reservation in response['Reservations']:
    for instance in reservation['Instances']:
        running_instances.append(instance['InstanceId'])
for instance_id in running_instances:
    print(instance_id)
    get_public_ip(instance_id)

# ### get image id of ubuntu ###
# filters = [
#     {'Name': 'name', 'Values': ['*GPU*PyTorch 1.13*Ubuntu*2023*']},
# ]
# response = ec2_client.describe_images(Filters=filters)
# for image in response['Images']:
#     print(f"Image ID: {image['ImageId']}, Name: {image['Name']}")

# ### List EC2 instances ###
# response = ec2_client.describe_instances()
# instances = []
# for reservation in response['Reservations']:
#     for instance in reservation['Instances']:
#         instance_info = {
#             'InstanceId': instance['InstanceId'],
#             'InstanceType': instance['InstanceType'],
#             'State': instance['State']['Name'],
#             'PublicIpAddress': instance.get('PublicIpAddress', 'N/A')
#         }
#         instances.append(instance_info)

# # Print instance details
# for instance in instances:
#     print(f"Instance ID: {instance['InstanceId']}")
#     print(f"Instance Type: {instance['InstanceType']}")
#     print(f"State: {instance['State']}")
#     print(f"Public IP: {instance['PublicIpAddress']}")
#     print("--------------------")

### to get unused ip
# response = ec2_client.describe_addresses()
# unused_ips = []
# # Check if each Elastic IP is associated with any instance
# for address in response['Addresses']:
#     if 'InstanceId' not in address:
#         unused_ips.append(address['PublicIp'])
# # Print the unused Elastic IPs
# print("Unused Elastic IPs:")
# for ip in unused_ips:
#     print(ip)

In [6]:
# ### create vpc that link to local network
# def create_vpc_and_subnet():
#     ec2_client = boto3.client('ec2')

#     # Create a VPC
#     response = ec2_client.create_vpc(CidrBlock='10.0.0.0/16')
#     vpc_id = response['Vpc']['VpcId']
#     print(f"VPC created with ID: {vpc_id}")

#     # Create a Subnet in the VPC
#     response = ec2_client.create_subnet(VpcId=vpc_id, CidrBlock='10.0.1.0/24')
#     subnet_id = response['Subnet']['SubnetId']
#     print(f"Subnet created with ID: {subnet_id}")

#     # Create a route table and associate it with the subnet
#     response = ec2_client.create_route_table(VpcId=vpc_id)
#     route_table_id = response['RouteTable']['RouteTableId']
#     print(f"Route Table created with ID: {route_table_id}")

#     # Replace with the CIDR block of local network
#     local_network_cidr = '192.168.1.0/24'

#     # Add a route to the route table for communication with your local network
#     ec2_client.create_route(
#         RouteTableId=route_table_id,
#         DestinationCidrBlock=local_network_cidr,
#         GatewayId='local'
#     )

#     # Associate the route table with the subnet
#     ec2_client.associate_route_table(RouteTableId=route_table_id, SubnetId=subnet_id)
#     return vpc_id, subnet_id, route_table_id

# # Call the function to create the VPC, Subnet, and route table
# vpc_id, subnet_id, route_table_id = create_vpc_and_subnet()

# print(f"VPC {vpc_id}, Subnet {subnet_id}, and Route Table {route_table_id} created successfully.")

In [ ]:
'''
find . | grep -E "(/__pycache__$|\.pyc$|\.pyo$)" | xargs rm -rf
ssh-keygen -q -b 2048 -t rsa -P "" -f ~/.ssh/ssh_key
ssh-copy-id -i ~/.ssh/ssh_key.pub chee-kean@192.168.96.107

ssh -i ~/.ssh/ec2-key-pair.pem -L 8050:localhost:8050 ubuntu@13.251.215.161
scp -i ~/.ssh/ec2-key-pair.pem -r /Volumes/dev/landing/mayo1/2023-01-10_16Hz_mayo1 ubuntu@13.251.215.161:/home/ubuntu/data/landing
ssh-copy-id -i ~/.ssh/ssh_key.pub gitlab@192.168.96.95
ssh-copy-id -i ~/.ssh/ssh_key.pub homepal@192.168.31.170

scp -u -i ~/.ssh/ec2-key-pair.pem -r /Volumes/dev/processed/ ubuntu@13.251.215.161:/home/ubuntu/data/
scp -i ~/.ssh/ssh_key.pub -r /Volumes/dev/processed/ chee-kean@192.168.96.107:/home/chee-kean/Dev/smartpassagelogic/app/weights/
ssh -i ~/.ssh/ec2-key-pair.pem ubuntu@13.213.58.68
scp -i ~/.ssh/ec2-key-pair.pem -r /Users/cheekeanchan/Dev/smartpassagelogic/ ubuntu@52.77.234.106:/home/ubuntu/
scp -i ~/.ssh/ec2-key-pair.pem -r /Volumes/dev/landing/mayo1* ubuntu@13.251.215.161:/home/ubuntu/data/landing
'''